Our aim is to predict whether or not a person has heart disease, using the dataset from https://www.kaggle.com/ronitf/heart-disease-uci.

We begin by importing some required packages, reading the data in and splitting data into X (features) and y (target) dataframes. 

In [2]:
import pandas as pd
import numpy as np
from scipy import stats


df=pd.read_csv(r'Desktop\KaggleDatasets\heart.csv') #read data into python

df.columns=["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","heart_disease"] #explicitly define column names and relabel target as heart_disease
df[(np.abs(stats.zscore(df)) < 3).all(axis=1)] #removes outliers, code from https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame
df.dropna()
df = pd.get_dummies(df,columns=['sex','cp','fbs','restecg','exang','slope','ca','thal'])

X=df.loc[:,df.columns!="heart_disease"] #let the feature dataframe contain every column of df, except the value we are predicting, heart_disease
y=df.loc[:,df.columns=="heart_disease"].values.ravel() #let the target dataframe contain only the value we are predicting, heart_disease








We select the best features to use for classification.

In [3]:
from sklearn.feature_selection import SelectKBest,f_classif

k=5 #k=5 leads to the best models
SKB = SelectKBest(f_classif,k=k) #select the 5 best features of X
SKB.fit_transform(X, y) #fit the data to SKB

X_best_indices=SKB.get_support(indices=True) 

best_predictors=[None]*(k) #k best predictors
for i in range(0,k):
    best_predictors[i]=X.columns[X_best_indices[i]] #create list of names of best predictors
       
print('The best predictors are',best_predictors)  #print names of best predictors 

X=X[best_predictors].copy() #use list to create new X dataframe


The best predictors are ['cp_0', 'exang_1', 'ca_0', 'thal_2', 'thal_3']


Next, we split the data into training and test sets and transform features using StandardScaler.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler


X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.20,random_state=42)                                            
                                                                                              
                                    
ss=StandardScaler()
ss.fit(X_train)
X_train=ss.transform(X_train)
X_test=ss.transform(X_test)


C:\Users\Natha\AppData\Local\Continuum\anaconda3\I2\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Natha\AppData\Local\Continuum\anaconda3\I2\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Natha\AppData\Local\Continuum\anaconda3\I2\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':


Since there may have been a nonlinear relationship between X and y, we tested various degrees of PolynomialFeatures.  However, logistic regression has the highest accuracy when the features have a degree of 1.

In [6]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score,f1_score  #polynomial features aren't useful in these case
from sklearn.preprocessing import PolynomialFeatures
for i in range(1,10):
    poly=PolynomialFeatures(i)
    poly.fit(X_train)
    X_train_poly=poly.transform(X_train)
    X_test_poly=poly.transform(X_test)
    Logreg=LogisticRegression(solver='lbfgs',max_iter=4000)
    Logreg.fit(X_train_poly,y_train)
    accscore=accuracy_score(y_test,Logreg.predict(X_test_poly))
    print('The accuracy score with X to the power of {} is {}'.format(i,accscore))

The accuracy score with X to the power of 1 is 0.9016393442622951
The accuracy score with X to the power of 2 is 0.8360655737704918
The accuracy score with X to the power of 3 is 0.8524590163934426
The accuracy score with X to the power of 4 is 0.8524590163934426
The accuracy score with X to the power of 5 is 0.8524590163934426
The accuracy score with X to the power of 6 is 0.8524590163934426
The accuracy score with X to the power of 7 is 0.8524590163934426
The accuracy score with X to the power of 8 is 0.8524590163934426
The accuracy score with X to the power of 9 is 0.8524590163934426


We define a logistic regression model and fit it to the data.  Since the classes are not very imbalanced, we check the accuracy of the model on the test set. We also use GridSearch to find the optimal hyperparameters.

In [21]:
from sklearn.metrics import classification_report,confusion_matrix
logreg=LogisticRegression(solver='lbfgs')
                         #define linear regression model
logreg.fit(X_train,y_train) #fit data to the linear regression model

y_pred_train=logreg.predict(X_train)
train_accuracy=accuracy_score(y_train, y_pred_train)

y_pred_test=logreg.predict(X_test)
test_accuracy=accuracy_score(y_test,y_pred_test)
print('The accuracies for logistic regression on the training and tests sets are, respectively, {} and {}'.format(train_accuracy,test_accuracy))


from sklearn.model_selection import GridSearchCV


grid={'penalty' : [#'l1', 
    'l2'], #https://towardsdatascience.com/logistic-regression-model-tuning-with-scikit-learn-part-1-425142e01af5
    'C' : [0.0001,0.001,0.01,0.1,1,10,100,1000],
    'solver' : [#'liblinear' 
                     'lbfgs'
               ]}

logreg_GS=GridSearchCV(estimator=logreg,param_grid=grid,cv=5,n_jobs=-1,refit=True).fit(X_train,y_train)

y_pred_train_GS=logreg_GS.predict(X_train)
train_accuracy_GS=accuracy_score(y_train, y_pred_train_GS)

y_pred_test_GS=logreg_GS.predict(X_test)
test_accuracy_GS=accuracy_score(y_test,y_pred_test_GS)
print('The accuracies for logistic regression from grid search on the training and tests sets are, respectively, {} and {}'.format(train_accuracy_GS,test_accuracy_GS))

print(classification_report(y_test,y_pred_test_GS))
print(confusion_matrix(y_test,y_pred_test_GS))


The accuracies for logistic regression on the training and tests sets are, respectively, 0.8388429752066116 and 0.9016393442622951
The accuracies for logistic regression from grid search on the training and tests sets are, respectively, 0.8388429752066116 and 0.9016393442622951
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        29
           1       0.91      0.91      0.91        32

   micro avg       0.90      0.90      0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61

[[26  3]
 [ 3 29]]


C:\Users\Natha\AppData\Local\Continuum\anaconda3\I2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


We use a support vector classifier in a similar way that we used logistic regression.

In [22]:
from sklearn import svm
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score,recall_score,precision_score, f1_score,classification_report,confusion_matrix
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

svc=svm.SVC(gamma='auto')
svc.fit(X_train,y_train)

y_pred_test=svc.predict(X_test)
#confusionmatrix=sklearn.metrics.confusion_matrix(y_test, y_pred_test)
accscore=accuracy_score(y_test, y_pred_test)
print(accscore)




parameters=[{'gamma':[1,0,0.1,0.001,0.0001],'C':[1,10,100,1000],'kernel':['linear','rbf']}]
svm=svm.SVC(gamma='auto')
svc_GS = GridSearchCV(svm, parameters,cv=5,iid='parameter',refit=True)

svc_GS.fit(X_train,y_train)

y_pred_test_GS=svc_GS.predict(X_test)
accscore_GS=accuracy_score(y_test, y_pred_test_GS)
print(accscore_GS)

print(classification_report(y_test,y_pred_test_GS))
print(confusion_matrix(y_test,y_pred_test_GS))


0.8524590163934426
0.9016393442622951
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        29
           1       0.91      0.91      0.91        32

   micro avg       0.90      0.90      0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61

[[26  3]
 [ 3 29]]


We defined and trained a random forest with default hyperparameters to predict target. We then performed an exhaustive randomised search using a grid with wide ranges of hyperparameter values (omitted). Then we repeat the search with a slightly narrowed down grid of values.  Note: this code will have a different output each time, due to the stochastic nature of boostrap sampling and randomised search.

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
#import matplotlib.pyplot as plt


rf=RandomForestClassifier(n_estimators=10)

rf.fit(X_train,y_train) 


#below, we use wide ranges of parameter values for our random search

random_grid = {'n_estimators': np.arange(10,70,1),
               'max_features': ['auto','sqrt'],
               'max_depth': np.arange(1,10,1),
               'min_samples_split': [2,3,4,5],
               'min_samples_leaf': [1,2,3,4],
               'bootstrap': [True,False]}

rf_RS = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1000, cv =5, verbose=True, iid=False,scoring='accuracy',n_jobs=-1) #does a randomised search and finds 'best' parameters
rf_RS.fit(X_train, y_train) #fit for each set of parameters


print(rf_RS.best_params_,rf_RS.best_score_)



Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1428 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 3928 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:   41.5s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=False, n_iter=1000, n_jobs=-1,
          param_distributions={'n_estimators': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69]), 'max_features': ['auto', 'sqrt'], 'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), 'min_samples_split': [2, 3, 4, 5], 

We use a random forest classifier with the 'best' hyperparameters found from one execution of the code above:



In [55]:
from sklearn.ensemble import RandomForestClassifier


rf_RS_best_params={'bootstrap': True, 'n_estimators': 15, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 5}

if 'bootstrap' in rf_RS_best_params:
       del rf_RS_best_params['bootstrap']
rf_RS=RandomForestClassifier(**rf_RS_best_params)
rf_RS.fit(X_train,y_train)
print('Accuracy score for rf on test data is',rf_RS.score(X_test,y_test))
print('Accuracy scores for rf_RS on train data and test data are,respectively {} and {}'.format(rf_RS.score(X_train,y_train),rf_RS.score(X_test,y_test)))



Accuracy score for rf on test data is 0.8524590163934426
Accuracy scores for rf_RS on train data and test data are,respectively 0.8553719008264463 and 0.8524590163934426


In [58]:
n_estimators =np.arange(10,45,1) #https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = np.arange(1,25,1)
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3,4,5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,3,4]
# Method of selecting samples for training each tree
#bootstrap = [True,False]
rf2=RandomForestClassifier(**rf_RS_best_params)
grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_GS=GridSearchCV(estimator=rf2,param_grid=grid,cv=5,n_jobs=-1,refit=True).fit(X_train,y_train)

print(rf_RS.best_params_,rf_GS.best_score_)
print('rf_GS score on training data is',accuracy_score(y_train,rf_GS.predict(X_train))) 
print('rf_GS score on test data is ',accuracy_score(y_test,rf_GS.predict(X_test)))


rf_GS score on training data is 0.8471074380165289
rf_GS score on test data is  0.9016393442622951


C:\Users\Natha\AppData\Local\Continuum\anaconda3\I2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


We use a random forest classifier with the 'best' hyperparameters found from one execution of the code above:



In [98]:
rf_GS_best_params={'bootstrap': True, 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 25}

if 'bootstrap' in rf_GS_best_params:
    del rf_GS_best_params['bootstrap']
        
rf_GS=RandomForestClassifier(**rf_GS_best_params)
rf_GS.fit(X_train,y_train)
print('rf_GS score on training data is',accuracy_score(y_train,rf_GS.predict(X_train))) 
print('rf_GS score on test data is ',accuracy_score(y_test,rf_GS.predict(X_test)))

rf_GS score on training data is 0.8512396694214877
rf_GS score on test data is  0.9016393442622951


We also use a gradient boosting classifier with the hyperparameters found from the gridsearch.  We found that 0.1 was the optimal learning rate, and used another gridsearch to find the optimal max depth and number of estimators.

In [104]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV    


rf_GS_best_params={'bootstrap': True, 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 25}


if 'bootstrap' in rf_GS_best_params:
    del rf_GS_best_params['bootstrap']
        
gbc=GradientBoostingClassifier(**rf_GS_best_params, learning_rate=0.1).fit(X_train,y_train)                         
                              

print('gbc score on training data is',accuracy_score(y_train,gbc.predict(X_train))) 
print('gbc score on test data is ',accuracy_score(y_test,gbc.predict(X_test)))


gbc_baseline=GradientBoostingClassifier().fit(X_train,y_train)

print('gbc baseline score on training data and test data are, respectively {} and {} '.format(gbc_baseline.score(X_train,y_train),gbc_baseline.score(X_test,y_test))) 


grid={'n_estimators':np.arange(1,42,1),
    'max_depth':[1,2,3,9]}


gbc_GS=GridSearchCV(estimator=gbc,param_grid=grid,cv=5,refit=True,n_jobs=-1,scoring='accuracy').fit(X_train,y_train)

print(gbc_GS.best_params_)
print('gbc_GS score on training data is',accuracy_score(y_train,gbc_GS.predict(X_train))) 
print('gbc_GS score on test data is ',accuracy_score(y_test,gbc_GS.predict(X_test)))



gbc score on training data is 0.8429752066115702
gbc score on test data is  0.8360655737704918
gbc baseline score on training data and test data are, respectively 0.8553719008264463 and 0.8524590163934426 
{'max_depth': 1, 'n_estimators': 24}
gbc_GS score on training data is 0.8264462809917356
gbc_GS score on test data is  0.9016393442622951


C:\Users\Natha\AppData\Local\Continuum\anaconda3\I2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Now we use a voting classifier to combine some of the models we have used so far and find that we have good accuray (90.16% on test data), although this is not an improvement on the previous models.

In [108]:
from sklearn.ensemble import  VotingClassifier

svc_GS.probability=True
vc= VotingClassifier(estimators=[
('logreg_GS', logreg_GS),
('svc_GS',svc_GS),
('rf_GS',rf_GS ),    
('gbc_GS',gbc)
                                    ]
,voting='hard',n_jobs=-1)
vc.fit(X_train,y_train)
votingclassifier(X_train,y_train)

y_pred_train=vc.predict(X_train)
y_pred_test=vc.predict(X_test)


print(accuracy_score(y_train,y_pred_train))  #0.8512396694214877

print(accuracy_score(y_test,y_pred_test))    #0.9016393442622951

0.8512396694214877
0.9016393442622951


In [109]:

print(classification_report(y_test,y_pred_test))


              precision    recall  f1-score   support

           0       0.90      0.90      0.90        29
           1       0.91      0.91      0.91        32

   micro avg       0.90      0.90      0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61

